In [15]:
import pandas as pd
import numpy as np
import os

In [16]:
model_name = 'P100_ms'
file_name = 'P100_ms'

In [17]:
DATA_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/P100_ms/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/P100_ms/")

## Dense Layer

In [4]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfDense

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [ ]:
dfDense.describe()

In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [49]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,activation_fct,use_bias,optimizer,gpu_count,timeUsed_median,timeUsed_min,timeUsed_max,timeUsed_std
0,36,472,7,15,13,3,0,32,0,False,2,2,1024.227285,855.402915,1033.930492,81.967417
1,29,301,5,18,12,3,0,32,2,True,4,2,410.584863,384.670655,467.865928,34.759683
2,11,472,6,18,11,4,0,32,2,False,5,2,327.342153,321.895369,359.779270,16.723317
3,44,290,4,16,18,4,1,32,2,False,4,2,442.387406,409.597452,538.596082,54.743784
4,14,259,3,10,20,3,0,32,3,False,2,2,101.360861,97.645609,106.621019,3.682245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2,269,2,6,28,3,0,32,2,True,1,2,27.363221,27.136429,29.645928,1.133322
496,27,449,3,5,8,4,1,32,2,False,2,2,257.121801,239.286049,259.158421,8.926695
497,44,376,2,10,22,2,0,32,1,False,4,2,786.388818,768.338490,825.920391,24.046795
498,24,290,5,6,19,4,0,32,0,False,3,2,111.184303,106.982764,131.789939,10.840465


In [50]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [51]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [35]:
print(dfConv.duplicated().sum())

0


In [37]:
dfConv.describe()

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu_count,...,opt_Adadelta,opt_Adagrad,opt_Adam,opt_Momentum,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.0,8000.000000,8000.0,...,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000
mean,32.449500,257.176250,3.978375,73.839625,71.423875,2.493500,0.499250,32.0,0.495875,2.0,...,0.167625,0.168750,0.163500,0.171875,0.163375,0.164875,0.24400,0.249250,0.250875,0.255875
std,18.638706,148.034395,2.013433,351.834749,336.273398,1.117749,0.500031,0.0,0.500014,0.0,...,0.373556,0.374554,0.369844,0.377295,0.369731,0.371091,0.42952,0.432606,0.433544,0.436379
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,32.0,0.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,16.000000,131.000000,2.000000,10.000000,10.000000,1.000000,0.000000,32.0,0.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,33.000000,258.500000,4.000000,19.000000,19.000000,2.000000,0.000000,32.0,0.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,49.000000,386.000000,6.000000,38.000000,37.000000,3.000000,1.000000,32.0,1.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000
max,64.000000,512.000000,7.000000,8605.000000,8508.000000,4.000000,1.000000,32.0,1.000000,2.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [52]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))

In [40]:
dfConv.to_csv(os.path.join(SAVE_DIR,'Data_convolution_%s.csv'%model_name), index = False)